In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from utils import loader
from utils import debugger 
from utils import preprocessing as pp 
from utils import misc
from utils import constant
import implementations as impl
import cost
import model

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = loader.load_csv_data(DATA_TRAIN_PATH)

In [72]:
to_replace = [(constant.UNDEF_VAL, 'most_frequent')]

augment_param = {
    'degree': [2],
    'add_bias' : True
}

x_split, y_split = pp.preprocess_jet_num(x, y, to_replace, do_normalise=False, augment_param=augment_param)

In [73]:
x_split[2].shape, y_split[2].shape

((72543, 58), (72543,))

In [74]:
data_part = []

nb_samples = []
for x_, y_ in zip(x_split, y_split):
    train_data, test_data = misc.random_data_split(y_, x_, k_fold=10)
    data = {'train': train_data, 'test': test_data}
    data_part.append(data)
    nb_samples.append(len(test_data[1]))

In [75]:
models = []
learning_param = { 'lambda_': 1e-6 }

acc_list = []

for idx in range(len(data_part)):
    x_tr, y_tr = data_part[idx]['train']
    nb_features = x_tr.shape[1]
    
    w_initial = np.random.uniform(0,1,size=nb_features) 
    model_ls = model.Model('ridge_regression', w_initial, learning_param, debug=True)
    
    model_ls.learn_weights(y_tr, x_tr)
    models.append(model_ls)
    
    tx_te, y_te = data_part[idx]['test']
    y_ = model_ls.predict(tx_te)
    acc_list.append(misc.accuracy(y_te, y_))

learning weights...
done.
learning weights...
done.
learning weights...
done.


In [76]:
acc_list

[0.8342508257431689, 0.7369099819448027, 0.7518610421836228]

In [77]:
avg_acc = np.dot(acc_list, nb_samples) / sum(nb_samples)
avg_acc

0.780151206048242

In [63]:
# Save weights
weights = [trained_model.w for trained_model in models]
file_name = 'weights.npy'
np.save(file_name, weights) # save

In [68]:
weights[2].shape

(59,)

In [69]:
weights_from_file = np.load(file_name) # load

array([array([-8.92467092e-01,  2.42072107e-03, -1.39349889e-02, -1.93397235e-02,
       -4.42529916e-04,  2.57448174e-01, -4.40895262e-04,  1.64388757e-02,
       -5.52563331e-01, -2.47942257e-04,  5.22258543e-03,  1.46145895e-03,
        3.20300570e-05,  1.11976432e-02,  1.74666105e-03, -6.95186448e-04,
       -7.71000745e-04, -2.45542530e-03,  1.51323987e-04, -2.50147125e-15,
       -8.51626118e-06,  7.54177135e-05,  2.95311033e-05, -1.41181802e-05,
        7.11987829e-02,  1.66355492e-05,  6.78147092e-05,  9.50671246e-02,
       -3.52168794e-02, -1.59230154e-04, -1.80315716e-02,  7.18470606e-04,
       -2.94548835e-04, -2.81313172e-02,  1.07487105e-03, -2.92193999e-06,
        1.21975115e-03,  1.20624671e-06,  0.00000000e+00]),
       array([-2.36504303e+00,  5.00676290e-03, -7.79622666e-03, -9.59765380e-03,
       -1.96411747e-03,  9.27180117e-01,  1.81445887e-03,  7.77361889e-03,
       -1.00737017e-01,  7.80630369e-02,  1.03673090e-02,  3.67150337e-03,
       -2.07089571e-03,  1